# Data Preparation & Cleaning
This notebook was in general used for cleaning and preparation of datasets, both for Hydrogen and the Benchmark

In [2]:
#Install necessaty libraries and packages
!pip install geopy
!pip install openpyxl
!pip install pandas
!pip install requests
!pip install chardet

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 667.8 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 3.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/11.6 MB 6.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.6 MB 10.5 MB/s eta 0:00:01
   -------- ----------

Here the initial data attained from the france hydrogen association was cleaned and saved.

In [20]:
import csv
import chardet

# List to store the cleaned data
clean_data = []

# Open the CSV file with 'rb' mode to read in binary mode
with open(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\webcrawler\data\france_member.csv', 'rb') as file:
    # Use chardet to detect the encoding of the file
    result = chardet.detect(file.read())
    encoding = result['encoding']

# Open the CSV file with the detected encoding
with open(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\webcrawler\data\france_member.csv', 'r', encoding=encoding) as file:
    reader = csv.reader(file)

    # Skip the header row
    next(reader)

    # Iterate through each row in the CSV file
    for row in reader:
        # Split the address by newline character(s)
        address_parts = row[2].replace('\r', '').split('\n')

        # Clean and join the address parts
        cleaned_address = ' '.join(address_parts)

        # Create a new row with the cleaned address
        new_row = [row[0], row[1], cleaned_address]

        # Add the new row to the clean_data list
        clean_data.append(new_row)

# Open a new CSV file to write the cleaned data
with open(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\webcrawler\data\france_member_cleaned.csv', 'w', newline='', encoding=encoding) as new_file:
    writer = csv.writer(new_file)

    # Write the header row
    writer.writerow(['Name', 'Representative', 'Address'])

    # Write the clean_data rows
    writer.writerows(clean_data)

   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   ------ -------------------------------- 30.7/199.4 kB 445.2 kB/s eta 0:00:01
   ---------------------------------- ----- 174.1/199.4 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 199.4/199.4 kB 1.7 MB/s eta 0:00:00


The following code was used to combine the data gathered from: https://www.wasserstoff-leitprojekte.de/leitprojekte

In [31]:
#load the distributed data into a pandas dataframe
import pandas as pd
df1 = pd.read_csv('../Gis/data/data-krTLV.csv')
df2 = pd.read_csv('../Gis/data/data-MDu07.csv')
df3 = pd.read_csv('../Gis/data/data-ZhFqn.csv')


In [32]:
#rename spatial columns to the same name
df1['Latitude'] = df1['LAT']
df1['Longitude'] = df1['LON']
df2['Latitude'] = df2['Lat']
df2['Longitude'] = df2['Lon']
df3['Latitude'] = df3['Lat']
df3['Longitude'] = df3['Lon']

In [33]:
#drop the columns that are not needed
df1.drop(['LAT', 'LON', 'Title'], axis=1, inplace=True)
df2.drop(['Lat', 'Lon', 'Title', 'Column 1'], axis=1, inplace=True)
df3.drop(['Lat', 'Lon'], axis=1, inplace=True)


In [34]:
#merge the dataframes and clean up the format of the columns. Delete duplicates
frames = [df1, df2, df3]
merge_df = pd.concat(frames)

#merge columns together for full address
merge_df['Full Address'] = merge_df['Straße'] + ', ' + merge_df['Ort'] + ', ' + merge_df['PLZ'].astype(str) + ', ' + merge_df['Land'].astype(str)

#delete duplicates in Column "Partner ohne Markdown"
merge_df = merge_df.drop_duplicates(subset='Partner ohne Markdown', keep='first')

merge_df
merge_df.to_excel('deutschland_projecte.xlsx', index=False)

# Create the dataframe for our benchmark data, which we got from the Amadeus database

In [2]:
#Create the dataframe for our benchmark data, which we will use to create a subset.
#The subset will have the same distribution of values in the 'Land' column as the original data.
#This will ensure that the subset is representative of the original data in terms of the distribution of countries.
import pandas as pd

df = pd.read_excel('locations/amadeus_benchmark_V1.xlsx')

# Define the desired number of rows in the subset
desired_subset_size = 30000

# Define the column you want to keep relatively consistent
desired_column = 'Land'

# Create a list of unique values in the desired column
unique_values = df[desired_column].unique()

# Calculate the current distribution of the desired column
current_distribution = df[desired_column].value_counts(normalize=True)

# Create an empty list to store the subset data
subset_data = []

# Sample rows from the original data based on the desired distribution
for value in unique_values:
    num_rows = int(desired_subset_size * current_distribution[value])
    subset_rows = df[df[desired_column] == value].sample(n=num_rows, replace=True)
    subset_data.append(subset_rows)

# Concatenate the sampled rows to create the final subset DataFrame
subset_df = pd.concat(subset_data, ignore_index=True)

# Display the original data
print("Original Data:")
print(df)

# Display the subset data
print("\nSubset Data:")
print(subset_df)

Original Data:
        Unnamed: 0                        Name des Unternehmens Länderkürzel  \
0              1.0                                    UNIPER SE           DE   
1              2.0                        ELECTRICITE DE FRANCE           FR   
2              3.0                                   ENI S.P.A.           IT   
3              4.0  GESTORE DEI SERVIZI ENERGETICI - GSE S.P.A.           IT   
4              5.0               VATTENFALL ENERGY TRADING GMBH           DE   
...            ...                                          ...          ...   
105755    105756.0          ZWÖLFTE SOLARKRAFTWERK SCHWEDT GMBH           DE   
105756    105757.0        ZWÖLFTE WINDKRAFTANLAGE GMBH & CO. KG           DE   
105757    105758.0                     ZWOLPOL PATRYK ZWOLIŃSKI           PL   
105758    105759.0                                   ZZ ECOTECH           BE   
105759    105760.0     ZZS ZEITZUSPAREN UG (HAFTUNGSBESCHRÄNKT)           DE   

                        

In [4]:
subset_df.to_excel('locations/amadeus_benchmark_V2.xlsx', index=False)